In [ ]:
import jax
from flax import nnx

from deeprte.model import modules
from deeprte.model.characteristics import Characteristics
from deeprte import input_pipeline

config = modules.DeepRTEConfig()
rng = nnx.Rngs(42)

In [ ]:
split = input_pipeline.Split.TRAIN
ds = input_pipeline.load(
    name="rte",
    split=split,
    split_percentage="20%",
    batch_sizes=[1],
    tfds_dir="/workspaces/deeprte/data/tfds",
    collocation_sizes=[120],
)
batch = next(ds)
jax.tree.map(lambda x: x.shape, batch)

## Test DeepRTE

In [ ]:
deeprte = modules.DeepRTE(config, rngs=rng)

nnx.display(deeprte)

### Params

In [ ]:
graphdef, params = nnx.split(deeprte)
jax.tree.reduce(lambda x, y: x + y, jax.tree.map(lambda x: x.size, params))

#### Case 1: train mode

In [ ]:
@jax.jit
def train_step(params, batch):
    module = nnx.merge(graphdef, params)
    module.set_attributes(low_memory=False)
    return module(batch)

In [ ]:
collocation_size = 128

subbatch = batch.copy()

subbatch["phase_coords"] = batch["phase_coords"][:, :collocation_size]
subbatch["scattering_kernel"] = batch["scattering_kernel"][:, :collocation_size]


%timeit prediction = train_step(params, subbatch)

#### Case 2: evaluation / inference mode

In [ ]:
@jax.jit
def eval_step(params, batch):
    module = nnx.merge(graphdef, params)
    module.set_attributes(low_memory=True)
    return module(batch)

In [ ]:
%%time

prediction = eval_step(params, batch)

print(prediction.shape)

## Test Green's function

In [ ]:
one_point_example = {}

single_example = jax.tree.map(lambda x: x[0], batch)
for k, v in single_example.items():
    if k in ["phase_coords", "scattering_kernel"]:
        one_point_example[k] = v[0]
    else:
        one_point_example[k] = v

jax.tree.map(lambda x: x.shape, one_point_example)

In [ ]:
green_fn = modules.GreenFunction(config=config, rngs=nnx.Rngs(0))

y = jax.vmap(green_fn, in_axes=(None, 0, None), out_axes=-1)(
    one_point_example["phase_coords"],
    one_point_example["boundary_coords"],
    one_point_example,
)

y.shape

## Test Attenuation module

In [ ]:
coord1 = jax.random.uniform(rng(), [5, 4])
coord2 = jax.random.uniform(rng(), [5, 4])
att_coeff = jax.random.uniform(rng(), [50, 2])

grid = jax.random.uniform(rng(), [50, 2])
char = Characteristics.from_tensor(grid)

attenuation = modules.Attenuation(config=config, rngs=nnx.Rngs(0))
jax.vmap(attenuation, in_axes=(0, 0, None, None))(coord1, coord2, att_coeff, char).shape